### Where vs. Having

- 'Where'절은 먼저 테이블의 개별 행에 적용되며 절의 조건을 충족시키는 행이 함께 그룹화됩니다. 
- 'Having'절은 결과 집합의 행에 적용됩니다. 

#### EX1)

In [3]:
!pip install mysqlclient

In [4]:
import MySQLdb

In [5]:
db = MySQLdb.connect(
    host="ip",
    user="id",
    passwd="pw",
    db="table_name",
    charset="utf8") # 한글 깨지면 charaset utf8로 설정

db

<_mysql.connection open to '52.78.107.76' at 0000021C2FB626D8>

In [6]:
Query = """
    Show Tables;
"""

In [7]:
curs = db.cursor()
curs.execute(Query)

8

In [8]:
rows = curs.fetchall()
rows

(('addr',),
 ('city',),
 ('cl_city',),
 ('code_count',),
 ('code_language',),
 ('code_name',),
 ('country',),
 ('countrylanguage',))

In [9]:
import pandas as pd

In [14]:
Query = """
    Select *
    From country

"""

In [16]:
df = pd.read_sql(Query, db)
df.tail(5)

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
234,YEM,Yemen,Asia,Middle East,527968.0,1918.0,18112000,59.8,6041.0,5729.0,Al-Yaman,Republic,Ali Abdallah Salih,1780.0,YE
235,YUG,Yugoslavia,Europe,Southern Europe,102173.0,1918.0,10640000,72.4,17000.0,NaN,Jugoslavija,Federal Republic,Vojislav Koštunica,1792.0,YU
236,ZAF,South Africa,Africa,Southern Africa,1221037.0,1910.0,40377000,51.1,116729.0,129092.0,South Africa,Republic,Thabo Mbeki,716.0,ZA
237,ZMB,Zambia,Africa,Eastern Africa,752618.0,1964.0,9169000,37.2,3377.0,3922.0,Zambia,Republic,Frederick Chiluba,3162.0,ZM
238,ZWE,Zimbabwe,Africa,Eastern Africa,390757.0,1980.0,11669000,37.8,5951.0,8670.0,Zimbabwe,Republic,Robert G. Mugabe,4068.0,ZW


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Code            239 non-null    object 
 1   Name            239 non-null    object 
 2   Continent       239 non-null    object 
 3   Region          239 non-null    object 
 4   SurfaceArea     239 non-null    float64
 5   IndepYear       192 non-null    float64
 6   Population      239 non-null    int64  
 7   LifeExpectancy  222 non-null    float64
 8   GNP             239 non-null    float64
 9   GNPOld          178 non-null    float64
 10  LocalName       239 non-null    object 
 11  GovernmentForm  239 non-null    object 
 12  HeadOfState     238 non-null    object 
 13  Capital         232 non-null    float64
 14  Code2           239 non-null    object 
dtypes: float64(6), int64(1), object(8)
memory usage: 28.1+ KB


#### 면적이 10000 제곱킬로미터 면적이고, 인구밀도가 200이상인 국가

In [27]:
Query = """
    Select Code, Name, round(population / SurfaceArea) as density 
    From country
    Where SurfaceArea >= 10000
    Having density >= 200
    Order by density desc
"""

In [28]:
df2 = pd.read_sql(Query, db)

In [30]:
df2.head(10)

,Code,Name,density
0,BGD,Bangladesh,897.0
1,TWN,Taiwan,615.0
2,KOR,South Korea,471.0
3,NLD,Netherlands,382.0
4,BEL,Belgium,336.0
5,JPN,Japan,335.0
6,LBN,Lebanon,316.0
7,IND,India,308.0
8,SLV,El Salvador,298.0
9,HTI,Haiti,296.0


#### df2의 국가들이 사용하고 있는 언어수가 5가지 이상인 나라 조회

In [33]:
Query_test = """
    Select *
    From countrylanguage
"""

In [34]:
df2 = pd.read_sql(Query_test, db)
df2

,CountryCode,Language,IsOfficial,Percentage
0,ABW,Dutch,T,5.3
1,ABW,English,F,9.5
2,ABW,Papiamento,F,76.7
3,ABW,Spanish,F,7.4
4,AFG,Balochi,F,0.9
...,...,...,...,...
979,ZMB,Tongan,F,11.0
980,ZWE,English,T,2.2
981,ZWE,Ndebele,F,16.2
982,ZWE,Nyanja,F,2.2


In [46]:
Query3 = """
    Select country.Name, count(Language) as language_count
    From (Select Code, Name, round(population / SurfaceArea) as density 
        From country
        Where SurfaceArea >= 10000
        Having density >= 200
        Order by density desc) as country
    Join countrylanguage
    on country.code = countrylanguage.countrycode
    Group by countrylanguage.countrycode
    having language_count >=5
    Order by language_count desc
"""

In [47]:
df3 = pd.read_sql(Query3, db)
df3

,Name,language_count
0,India,12
1,Philippines,10
2,Vietnam,9
3,Bangladesh,7
4,Taiwan,6
5,Germany,6
6,Japan,6
7,Belgium,6
